<a href="https://colab.research.google.com/github/otviio/Alura_desafio/blob/main/desafioAlura_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O desafio tem o intuito de testar a capacidade do Gemini identificar elementos dentro de um determinado vídeo.

Vídeo esse que é um que eu mesmo gravei fazendo tarefas no jogo Stardew Valley, um jogo que atualmente estou jogando muito. Daí pensei, por que não testar a capacidade do Gemini de analizar vídeos e pedir para descrever o que está acontecendo.

Instalando o SDK do Gemini

In [1]:
!pip install -U -q google-generativeai

Importando o generative AI

In [2]:
import google.generativeai as genai

Configurando a API Key do google

In [3]:
from google.colab import userdata

key=userdata.get('SECRET')
genai.configure(api_key=key)

## Salvando o video em uma variável.

Veja o video [aqui](https://drive.google.com/file/d/1zHzt1t2OjZjkCsb4u22ttRSVup853xaP/view?usp=drive_link).

In [4]:
video_file_name = "/content/drive/MyDrive/AluraDesafio/video01.mp4"

## É aqui onde tudo começa.

Como a API Gemini atualmente não oferece suporte direto a arquivos de vídeo. Em vez disso, podemos fornecer uma série de timestamps e arquivos de imagem.

---


Neste código fornecido pelo github do Google Gemini, [cookbook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb), ele extrai o video em imagens (1 frame por segundo) para uma pasta do meu drive chamada frames. Dando 61 frames na extração.

In [5]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/drive/MyDrive/AluraDesafio/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

Extracting /content/drive/MyDrive/AluraDesafio/video01.mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 61 frames


## Neste código, ele faz o upload dos frames para a API.

No código original ele faz upload de 10 dos 600 frames da extração, como temos poucos frames (e também para melhorar a resposta do Gemini um pouco) eu coloquei para fazer upload de todos os arquivos.

In [6]:
import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
    return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Set full_video to True to upload the whole video
full_video = True

uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload[40:50]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

Uploading 61 files. This might take a bit...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:00.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:01.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:02.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:03.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:04.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:05.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:06.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:07.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:08.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:09.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:10.jpg...
Uploading: /content/drive/MyDri

## Listando os arquivos que foram carregados

In [7]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/v8elkdcp0u6g
https://generativelanguage.googleapis.com/v1beta/files/tlz8j29kvihy
https://generativelanguage.googleapis.com/v1beta/files/dql4j6m1sgrf
https://generativelanguage.googleapis.com/v1beta/files/3nr2ph046591
https://generativelanguage.googleapis.com/v1beta/files/8yb13m4gn9yn
https://generativelanguage.googleapis.com/v1beta/files/4e2q3zmd9jtr
https://generativelanguage.googleapis.com/v1beta/files/p9nrhce6cmv4
https://generativelanguage.googleapis.com/v1beta/files/ydbjyspylvn9
https://generativelanguage.googleapis.com/v1beta/files/9ibbpc5igjb3
https://generativelanguage.googleapis.com/v1beta/files/8qkaov63f6b3
https://generativelanguage.googleapis.com/v1beta/files/19a3eyf5bsfv
https://generativelanguage.googleapis.com/v1beta/files/g0wd6s44udyk
https://generativelanguage.googleapis.com/v1beta/files/n0f3od29e25n
https://generativelanguage.googleapis.com/v1beta/files/533l5s7lt5gu
https://generativelanguage.googleapis.com/v1beta

## Configuração do modelo Gemini 1.5 Pro.

In [8]:
# Set the model to Gemini 1.5 Pro.
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

## Criando o prompt

Aqui o Gemini acertou quase tudo praticamente, errando apenas 3 coisas, os espantalhos que ele diz ser outros dois personagens, mas pensando bem, não acho que está totalmente errado nesta parte, já que da para "conversar" com o espantalho (ele diz quantos corvos espantou). O outro erro foi ele citar que tem silos vermelhos (provavelmente seria as geléias que o personagem coleta). E também tem o erro que ele cismou, que é outra cama que ele fica dizendo que é um berço, nos meus outros testes tentei colocar no prompt que não tinha berço, mas ele colocava do mesmo jeito.

In [9]:
# Create the prompt.
prompt = "Descreva o video. Faça o texto com quebra de linha"


# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

O vídeo começa com uma tela preta com a mensagem "Carregando..." em um pergaminho amarelo.

Em seguida, o jogo Stardew Valley é carregado, mostrando o interior de uma casa. Um personagem com chapéu de palha e roupa azul sai do quarto e caminha pela casa.

O jogador abre o inventário e passa por suas ferramentas: vara de fibra de vidro, espada de obsidiana, picareta de aço, enxada, machado de cobre, vara de irídio e regador.

O personagem sai da casa e entra em sua fazenda. A fazenda tem um celeiro, um galinheiro, uma horta cercada, árvores, um lago e uma área em construção.

Dois outros personagens, um com chapéu preto e outro com chapéu de palha, estão na horta. O jogador se junta a eles.

O jogador sai da horta e caminha em direção a uma fileira de silos vermelhos. Ele passa por uma árvore rosa florida e chega a uma área em construção próxima a um penhasco.

O jogador coloca dois silos na área em construção, totalizando dez silos. Em seguida, ele retorna à casa, passando novamente pe

**Aqui eu peço para ele descrever dentro da casa. O Gemini acerta tudo, mas continuando citando o berço**

In [16]:
# Create the prompt.
prompt = "Como é o ambiente dentro da casa. Faça quebra de linha."


# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

A casa é dividida em dois ambientes:

- À esquerda, uma cozinha com piso quadriculado em preto e branco, armários, pia, geladeira, fogão e lixeira. Há também uma mesa de jantar com quatro cadeiras e um vaso de flores. Duas plantas enfeitam o ambiente.

- À direita, um quarto com piso rosa claro, uma cama de casal, um tapete marrom, uma lareira, uma televisão, um berço, uma cômoda e um urso de pelúcia azul. Duas pinturas decoram as paredes.



**Agora eu peço para ele descrever a plantação. Ele acerta tudo excepcionalmente, aqui ele até acerta os espantalhos.**

In [17]:
# Create the prompt.
prompt = "Descreva a plantação. Faça o texto com quebra de linha"


# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

A plantação é cercada por uma cerca de madeira escura.
Ela é retangular e está localizada próxima à casa do jogador.
Há um espantalho no canto superior esquerdo da plantação e outro no canto inferior direito.
A plantação parece estar em bom estado, com terra arada e plantas saudáveis crescendo.
Há uma árvore verde alta no canto inferior esquerdo da plantação e uma árvore rosa florida no canto superior esquerdo.
Há um pequeno lago no canto inferior direito da plantação.
Há um caminho de terra que leva da casa do jogador até a plantação.
Há várias árvores e arbustos ao redor da plantação.
No canto superior esquerdo da tela, há uma área com vários barris de madeira.
No canto inferior esquerdo da tela, há uma área com uma construção inacabada.
No canto inferior direito da tela, há uma área com várias caixas de madeira.


**Aqui eu peço para ele tentar acertar o que o personagem coleta no jarro/barril de conserva. No prompt tem uma dica por causa que nos testes que fiz, o Gemini sempre errava, mas com a dica ele acerta.**

In [12]:
# Create the prompt.
prompt = "O que ele coleta no jarro de conserva? Dica: é geleia. Faça o texto com quebra de linha"


# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

Ele coleta **Geleia de Oxicoco** no jarro de conserva. 



## Agora eu irei deletar os frames que fiz upload para a API e fazer outro teste.

In [18]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files:
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

Deleting 61 images. This might take a bit...
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:00.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/rjy18hkdlzet
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:01.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/3ffzxq6e27sf
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:02.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/e6ilomepwqel
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:03.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/vh0nnagq2mjh
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:04.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/m1m2gtb5j8hy
Deleted /content/drive/MyDrive/AluraDesafio/frames/video01_mp4_frame00:05.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/z7ukebp6ea12
Deleted /content/drive/MyDrive/AluraD

## O outro teste começa aqui

Mesma coisa de antes, guardando o video numa variável, extraindo e carregando os frames na API. Mas tem uma coisa diferente, este vídeo é o mesmo de antes, porém, quase não dá para entender o que está acontecendo. O [vídeo](https://drive.google.com/file/d/1XB4Tlm-fft3IEvlD8QY_pnftATaFNy9c/view?usp=drive_link). Um dos [frames](https://drive.google.com/file/d/15Be4OOdg-WLm2qGoREvghDkI1gHz8ejM/view?usp=drive_link) que o Gemini salvou

In [19]:
video_file_name = "/content/drive/MyDrive/AluraDesafio/video02.mp4"

In [20]:
FRAME_EXTRACTION_DIRECTORY = "/content/drive/MyDrive/AluraDesafio/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

Extracting /content/drive/MyDrive/AluraDesafio/video02.mp4 at 1 frame per second. This might take a bit...
Completed video frame extraction!

Extracted: 61 frames


In [21]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
    return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Set full_video to True to upload the whole video
full_video = True

uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload[40:50]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

Uploading 61 files. This might take a bit...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:00.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:01.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:02.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:03.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:04.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:05.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:06.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:07.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:08.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:09.jpg...
Uploading: /content/drive/MyDrive/AluraDesafio/frames/video02_mp4_frame00:10.jpg...
Uploading: /content/drive/MyDri

E aqui, chegando ao final do teste, foi onde eu realmente me surpreendi, o Gemini acertou tudo que esta acontecendo no vídeo, mesmo vídeo estando numa qualidade horrível.

In [24]:
# Create the prompt.
prompt = "Descreva o video. Faça o texto com quebra de linha"


# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

O vídeo começa com uma tela de carregamento do jogo Stardew Valley.

Em seguida, o jogo é iniciado e vemos o personagem principal, um fazendeiro, dentro de sua casa. O fazendeiro está em uma sala com piso quadriculado em preto e branco, uma mesa de jantar, uma cozinha e uma porta que leva a outra sala.

O fazendeiro abre o menu de ferramentas e percorre as opções: espada de madeira, picareta de aço, enxada, machado de cobre, vara de brilho e regador.

Em seguida, o fazendeiro sai de casa e vemos a fazenda vista de cima. A fazenda tem uma casa, um celeiro, um galinheiro, um lago e uma área para plantar. O fazendeiro caminha pela fazenda e observa as plantações.

O fazendeiro continua caminhando pela fazenda, passando por árvores e cercas.

O vídeo termina com o fazendeiro voltando para dentro de casa.
